In [1]:

import pickle
import numpy as np
import pandas as pd 
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error
import joblib



XGBoostError: XGBoost Library (libxgboost.so) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libgomp.so for UNIX-like OSes)
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['libgomp.so.1: cannot open shared object file: No such file or directory']


In [2]:
pip install xgboost==1.0.2


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
demand = pd.read_csv("demand_hourly.csv")
tss = TimeSeriesSplit(n_splits=5, test_size=24*60, gap=24)
df = demand.sort_values(by=['start_time_year', 'start_time_month', 
                            'start_time_day', 'start_time_hour'])
df = df.drop(columns=['Unnamed: 0', 'duration_sec']).reset_index(drop=True)

standard_stations = pd.read_csv("stations_with_clusters.csv")
weather_data = pd.read_csv("weather_data.csv")

df = df.merge(standard_stations[['station_name', 'clusters']].rename(columns={
    "station_name":'start_station_name'}), on=["start_station_name"])

df = df[df.groupby('start_station_name')['start_station_name'].transform('size') >= 1000]

latest_trips = df.groupby('start_station_name').nth(-1)
earliest_trips = df.groupby('start_station_name').nth(0)


recently_operational_stations = latest_trips[(latest_trips['start_time_year'] == 2022) & 
                                             (latest_trips['start_time_month'] >= 8)].index

stations_operational_since_2021 = earliest_trips[(earliest_trips['start_time_year'] == 2021)].index
df = df[df['start_station_name'].isin(stations_operational_since_2021)].reset_index(drop=True)

column_name = "start_time"
conversion_dict_daily = dict(year= df[f'{column_name}_year'],
                           month=df[f'{column_name}_month'],
                           day=  df[f'{column_name}_day'],
                           hour=  df[f'{column_name}_hour']
                        )
df['time'] = pd.to_datetime(conversion_dict_daily).astype(str)

df = df.merge(weather_data[['temp', 'dwpt', "rhum", "prcp", "wdir", "wspd", "pres", "coco", "centroid", 'time']].rename(columns={
    "centroid":"clusters"
}), on=['clusters', "time"])

df.loc[df['prcp'].isna(), 'prcp'] = 0.0
df.loc[df['pres'].isna(), 'pres'] = df['pres'].median()
df.loc[df['coco'].isna(), 'coco'] = df['coco'].mode()[0]

In [ ]:
def add_lags(df, target, identifier):
    df_res = pd.DataFrame()
    print(target)
    for ii in df[identifier].unique():
        df_current = df[df["start_station_name"]==ii]
        df_current.index = df_current['time']
        df_current.index = pd.to_datetime(df_current.index)
        target_map = df_current[target].to_dict()
        df_current[f"{target}_lag_1_h"] = (df_current.index - pd.Timedelta('1 hours')).map(target_map)
        df_current[f"{target}_lag_2_h"] = (df_current.index - pd.Timedelta('2 hours')).map(target_map)
        df_current[f"{target}_lag_24_h"] = (df_current.index - pd.Timedelta('24 hours')).map(target_map)
        if target == "demand":
            df_current[f"{target}_lag_1_h"] = df_current[f"{target}_lag_1_h"].fillna(0)
            df_current[f"{target}_lag_2_h"] = df_current[f"{target}_lag_2_h"].fillna(0)
            df_current[f"{target}_lag_24_h"] = df_current[f"{target}_lag_24_h"].fillna(0)
        else:
            df_current[f"{target}_lag_1_h"] = df_current[f"{target}_lag_1_h"].interpolate().fillna(0)
            df_current[f"{target}_lag_2_h"] = df_current[f"{target}_lag_2_h"].interpolate().fillna(0)
            df_current[f"{target}_lag_24_h"] = df_current[f"{target}_lag_24_h"].interpolate().fillna(0)
        df_res = df_res.append(df_current)
    return df_res

In [ ]:
df = add_lags(df, "demand", identifier='start_station_name')
df = add_lags(df, "temp", identifier='start_station_name')
df = add_lags(df, "prcp", identifier='start_station_name')
df = add_lags(df, "rhum", identifier='start_station_name')
df = add_lags(df, "wspd", identifier='start_station_name')

df = df.sort_values(by=['start_time_year', 'start_time_month', 
                        'start_time_day', 'start_time_hour']).drop(
    columns=['time']).reset_index(drop=True)

In [ ]:
df.info()

In [ ]:
for i in df.columns:
    if 'lag' in i:
        print((df[(df['start_station_name'] == "1st St at Folsom St")][i].interpolate().isna()))

In [ ]:
for i in df.columns:
    print(i)
    print(sum(df[i].isna()))

In [ ]:
del demand

In [ ]:
df.tail()

In [ ]:

df = df.sort_index()

fig, axs = plt.subplots(5, 1, figsize=(15, 15), sharex=True)

fold = 0
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]
    train['demand'].plot(ax=axs[fold],
                          label='Training Set',
                          title=f'Data Train/Test Split Fold {fold}')
    test['demand'].plot(ax=axs[fold],
                         label='Test Set')
    axs[fold].axvline(test.index.min(), color='black', ls='--')
    fold += 1
plt.show()

In [ ]:
tss = TimeSeriesSplit(n_splits=5, test_size=24*5*df['start_station_name'].nunique(), gap=24)
df = df.sort_index()

fold = 0
preds = pd.DataFrame()
rmse = []
mape = []
le = preprocessing.LabelEncoder()
scaler = MinMaxScaler(feature_range=(1, 2))
SCALER_FEATURES = ["start_time_year", "start_time_month", 
                   "start_time_day", "start_time_hour", 
                   "start_time_week", "start_time_quarter",
                   "start_time_dayofweek",
                   "temp", "dwpt", "rhum", "prcp", "wdir",
                   "wspd", "pres","coco",
                   "demand_lag_1_h", "demand_lag_2_h", 
                   "demand_lag_24_h", "temp_lag_1_h", 
                   "temp_lag_2_h", "temp_lag_24_h",
                   "prcp_lag_1_h", "prcp_lag_2_h",
                   "prcp_lag_24_h", "rhum_lag_1_h",
                   "rhum_lag_2_h", "rhum_lag_24_h", "wspd_lag_1_h", 
                   "wspd_lag_2_h","wspd_lag_24_h"]
target_scaler = MinMaxScaler(feature_range=(1, 2))
df[SCALER_FEATURES] = scaler.fit_transform(df[SCALER_FEATURES])
df[['demand']] = target_scaler.fit_transform(df[['demand']])
df['start_station_name'] = le.fit_transform(df['start_station_name'])
df['is_holiday'] = df['is_holiday'].astype(int)


In [ ]:
24*5*df['start_station_name'].nunique()

In [ ]:
24*10*df['start_station_name'].nunique()

In [ ]:

for train_idx, val_idx in tss.split(df):

    train = df.iloc[train_idx]
    test = df.iloc[val_idx]


    FEATURES = ["start_station_name", 
                "start_time_year",
                "start_time_month",
                "start_time_day",
                "start_time_hour",
                "start_time_week",
                "start_time_quarter",
                "start_time_dayofweek",
                "is_holiday",
                "clusters",
                "temp",
                "dwpt",
                "rhum",
                "prcp",
                "wdir",
                "wspd",
                "pres",
                "coco",
                "demand_lag_1_h",
                "demand_lag_2_h",
                "demand_lag_24_h",
                "temp_lag_1_h",
                "temp_lag_2_h",
                "temp_lag_24_h",
                "prcp_lag_1_h",
                "prcp_lag_2_h",
                "prcp_lag_24_h",
                "rhum_lag_1_h",
                "rhum_lag_2_h",
                "rhum_lag_24_h",
                "wspd_lag_1_h",
                "wspd_lag_2_h",
                "wspd_lag_24_h"
                ]
    TARGET = 'demand'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgboost.XGBRegressor()
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    preds_out = X_test.copy()
    preds_out['actual_demand'] = y_test
    preds_out['pred'] = y_pred
    preds = preds.append(preds_out)
    rmse_score = np.sqrt(mean_squared_error(y_test, y_pred))
    mape_score = mean_absolute_percentage_error(y_test, y_pred)
    rmse.append(rmse_score)
    mape.append(mape_score)
    print(rmse_score)
    print(mape_score)
    print(preds.head())
    print(preds.tail())
    
print(f"Mean RMSE: {np.mean(rmse)}")
print(f"Mean MAPE: {np.mean(mape)}")
del df
del X_train
del y_train
del X_test

In [14]:
suf = "_dec_2022_weather_lags"

#preds.to_csv(f"test_predictions{suf}.csv")
#del preds
#
#filename = f'demand_model{suf}.sav'
#joblib.dump(reg, filename)
#del reg

filename = f'target_scaler{suf}.sav'
joblib.dump(target_scaler, filename)
del target_scaler

filename = f'scaler{suf}.sav'
joblib.dump(scaler, filename)
del scaler

filename = f'label_encoder{suf}.sav'
joblib.dump(le, filename)
del le


In [15]:
#preds['demand'] = preds['actual_demand']
#preds[SCALER_FEATURES] = scaler.inverse_transform(preds[SCALER_FEATURES])